# Chisel Project

Chisel stands for **C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage. That means it is a DSL in Scala, allowing you to take advantage of both Scala and Chisel programming within the same code. 

Here we get familar with basic chisel concepts and progressively build a very simple CPU in chisel and run assembly code on it using the scala compiler we developed in the previous tutorial.

### Setup

The following cell downloads the dependencies needed for Chisel.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.tester._
import chisel3.tester.RawTester.test

Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module`, `RegisterFile`, that has 32 32-bit registers, a configurable number of read ports and a write port. 
Because Chisel Modules are normal Scala classes, we can use the power of Scala's class constructors to parameterize the elaboration of our design.

In [ ]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool()) //Write enable
        val waddr = Input(UInt(5.W))  //Write address
        val wdata = Input(UInt(32.W)) //Write data
        val raddr = Input(Vec(readPorts, UInt(5.W)))  //Read address
        val rdata = Output(Vec(readPorts, UInt(32.W))) //Read data
    })
    
    // A Register of a vector of UInts
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))
    
    when (io.wen) {
        reg(io.waddr) := io.wdata //connect the input write data to the corresponding register
    }
    
    //According to the number of readPorts make the connections for raddr and rdata
    for (i <- 0 until readPorts) { 
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U //x0 is hardwired to 0 (risc-v spec requirement)
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }
}

//Print the verilog
// println(getVerilog(new RegisterFile()))

* `Module` is a built-in Chisel class that all hardware modules must extend.
* We declare all our input and output ports in a special io val. It must be called io and be an IO object or instance, which requires something of the form IO(_instantiated_bundle_).

### Testing Your Hardware
No hardware module or generator should be complete without a tester. Chisel has built-in test features that you will explore. The following example is a Chisel test harness that passes values to an instance of `RegisterFile`'s input port, and checks that the same value is seen on readback.

In [ ]:
// Scala Code: `test` runs the unit test. 
// test takes a user Module and has a code block that applies pokes and expects to the 
// circuit under test (c)
test(new RegisterFile(2) ) { c =>
  //Helper function to read and check the value received
  def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
    c.io.raddr(port).poke(addr.U)
    c.io.rdata(port).expect(value.U)
  }
  //Helper function to write to a particular register
  def write(addr: Int, value: Int): Unit = {
    c.io.wen.poke(true.B)
    c.io.wdata.poke(value.U)
    c.io.waddr.poke(addr.U)
    c.clock.step(1)
    c.io.wen.poke(false.B)
  }
  // everything should be 0 on init
  for (i <- 0 until 32) {
    readExpect(i, 0, port = 0)
    readExpect(i, 0, port = 1)
  }

  // write 5 * addr + 3
  for (i <- 0 until 32) {
    write(i, 5 * i + 3)
  }

  // check that the writes worked
  for (i <- 0 until 32) {
    readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

The test accepts a `RegisterFile` module, assigns values to the module's inputs, and checks its outputs. To set an input, we call `poke`. To check an output, we call `expect`. If we don't want to compare the output to an expected value (no assertion), we can `peek` the output instead.

If all expect statements are true, then our boilerplate code will return pass.

In [ ]:
//Full code put together to form the assembler class
case class RVAssembler() {
    val supported_ops = Array("addi", "add", "sub", "mul")
    val reg_map = Map("x0"  -> 0,  "x1"  -> 1,  "x2"  -> 2,  "x3"  -> 3,
                      "x4"  -> 4,  "x5"  -> 5,  "x6"  -> 6,  "x7"  -> 7,
                      "x8"  -> 8,  "x9"  -> 9,  "x10" -> 10, "x11" -> 11,
                      "x12" -> 12, "x13" -> 13, "x14" -> 14, "x15" -> 15,
                      "x16" -> 16, "x17" -> 17, "x18" -> 18, "x19" -> 19,
                      "x20" -> 20, "x21" -> 21, "x22" -> 22, "x23" -> 23,
                      "x24" -> 24, "x25" -> 25, "x26" -> 26, "x27" -> 27,
                      "x28" -> 28, "x29" -> 29, "x30" -> 30, "x31" -> 31
                     )
    val supported_regs = reg_map.keys.toList

    def assemble(asm_string: String): Int = {
        println(s"Convert $asm_string assembly to machine code")
        
        //Get the opcode
        val opcode = asm_string.split(" ")(0).toLowerCase().stripSuffix(",")
        require(supported_ops.contains(opcode))
        
        //Get the operands
        val operands = List((asm_string.split(" ")(1).stripSuffix(",")), (asm_string.split(" ")(2).stripSuffix(",")), (asm_string.split(" ")(3)))
        require(operands.size == 3)
        
        //Based on the opcode take different actions
        var inst: Int = 0
        opcode match {
            case "addi" => {
                check_i_type(operands)
                inst = ITypeInstr(operands(1), operands(0), operands(2))
            }
            case "add"  => {
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x00)
            }
            case "sub"  => { 
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x20)
            }
            case "mul"  => {
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x01)
            }
            case _ => throw new Exception("Unsupported instruction") 
        }
        return inst
    }
    
    def check_i_type(operands: List[String]) = {
        val imm_val = operands(2).toInt //Convert string to Integer
        require(imm_val >= -2048 && imm_val <= 2047, "Immediate value out of range") //Immediate is limited to 12bits
        require(supported_regs.contains(operands(0)), s"{operands(0)} Illegal register value")
        require(supported_regs.contains(operands(1)), s"{operands(1)} Illegal register value")
    }
    
    def check_r_type(operands: List[String]) = {
        require(supported_regs.contains(operands(0)), s"{operands(0)} Illegal register value")
        require(supported_regs.contains(operands(1)), s"{operands(1)} Illegal register value")
        require(supported_regs.contains(operands(2)), s"{operands(2)} Illegal register value")
    }
    
    def ITypeInstr(rs1: String, rd: String, imm: String): Int = {
        val imm_val = imm.toInt
        val bin_val: Int = imm_val << 20 | reg_map(rs1) << 15 | 0 << 12 | reg_map(rd) << 7 | 0x13
        return bin_val
    }
    
    def RTypeInstr(rs1: String, rs2: String, rd: String, funct7: Int): Int = {
        val bin_val: Int = funct7 << 25 | reg_map(rs2) << 20 | reg_map(rs1) << 15 | 0 << 12 | reg_map(rd) << 7 | 0x33
        return bin_val
    }
}


In [ ]:
class CrayRV32() extends Module {
    val io = IO(new Bundle {
        val i_bus = Input(UInt(32.W))  //32-bit instruction input
        val illegal_instr = Output(Bool())  //Raised when a illegal instruction is encountered
        val raddr = Input(UInt(2.W))  //Read port of register file for testbench
        val rdata = Output(UInt(32.W))  //Read port of register file for testbench
    })
    
    //Instantiate the register file module with 3 read ports
    val reg_file = Module(RegisterFile(3))
    
    //Connect one of the read ports to output
    io.rdata := reg_file.io.rdata(0)
    reg_file.io.raddr(0) := io.raddr
    
    //Create slices of the 32-bit instruction input according to the instruction bitmap
    val opcode = io.i_bus(6, 0)
    val rd     = io.i_bus(11, 7)
    val funct3 = io.i_bus(14, 12)
    val rs1    = io.i_bus(19, 15)
    val rs2    = io.i_bus(24, 20)
    val funct7 = io.i_bus(31, 25)
    val imm    = io.i_bus(31, 20)
    
//     printf("instr: 0x%x, opcode = 0x%x\n",io.i_bus, opcode)
    
    reg_file.io.waddr := rd
    
    when(opcode === 0x33.U) {  //Handle R-type instructions //How to put this in a clocked always@??
//         printf(p"R Type rd:$rd rs1:$rs1 rs2:$rs2\n")
        //Set the register read addresses
        reg_file.io.raddr(1) := rs1 
        reg_file.io.raddr(2) := rs2
        //Set defaults when none of the cases match below
        reg_file.io.wen      := false.B
        io.illegal_instr     := true.B
        reg_file.io.wdata    := 0.U
        switch(funct7) {  //How to put default case??
            is(0x00.U) { //ADD
                printf(p"ADD  [$rd] = [$rs1] + [$rs2]\n") //Print during simulation Scala style formatting
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) + reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
            is(0x01.U) { //MUL
                printf("MUL  [%d] = [%d] * [%d]\n", rd, rs1, rs2) //Print during simulation C style formatting
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) * reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
            is(0x20.U) { //SUB
                printf("SUB  [%d] = [%d] - [%d]\n", rd, rs1, rs2)
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) - reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
        }
    }.elsewhen(opcode === 0x13.U) { //ADDI
        io.illegal_instr := false.B
        printf("ADDI [%d] = [%d] + %d\n", rd, rs1, imm)
        reg_file.io.raddr(1) := rs1
        reg_file.io.raddr(2) := 0.U
        reg_file.io.wen      := true.B
        reg_file.io.wdata    := imm + reg_file.io.rdata(1)
    }.otherwise {
        io.illegal_instr := true.B
        reg_file.io.wen  := false.B
        reg_file.io.raddr(1) := 0.U
        reg_file.io.raddr(2) := 0.U
        reg_file.io.wdata    := 0.U
    }   
}

//Print the verilog
// println(getVerilog(new CrayRV32()))

In [ ]:
//Simple sanity check
test(new CrayRV32()) { c =>
    val assembler = RVAssembler()
    
    val instr_list = Seq(
        "addi x1, x0, 30",
        "addi x2, x0, 10",
        "addi x3, x0, 10",
        "add x3, x1, x2",
        "sub x3, x1, x2",
        "mul x3, x1, x2"
    )
    
    //Golden values to check after execution of each instruction
    val reg_golden = Seq(
        //reg_num, expected value
        Seq(1, 30),
        Seq(2, 10),
        Seq(3, 10),
        Seq(3, 40),
        Seq(3, 20),
        Seq(3, 300)
    )
    
    def readExpect(addr: Int, value: Int): Unit = {
        c.io.raddr.poke(addr.U)
//         println(s"${c.io.rdata.peek().litValue.toInt} ")
        c.io.rdata.expect(value.U)
    }
    
    def check_golden(indx: Int): Unit = {
        readExpect(reg_golden(indx)(0), reg_golden(indx)(1))
    }
    
    //Iterate over the instruction list and check with golden value on every step    
    instr_list.zipWithIndex.foreach{
        case (instr, idx) => {
            val inst = assembler.assemble(instr)
            c.io.i_bus.poke(inst.U)
            c.io.illegal_instr.expect(false.B)
            c.clock.step(1)
            check_golden(idx)
        }
    }
//     var idx = 0
//     for (instr <- instr_list) {
//         val inst_enc = compiler.compile(instr)
//         c.io.i_bus.poke(inst_enc.U)
//         c.clock.step(1)
//         check_golden(idx)
//         idx = idx + 1
//     }
}

